# Implement function calling with the Granite-3-8B-Instruct model in Python with WatsonX


**Authors:** Erika Russi, Anna Gutowska, Jess Bozorg

In this tutorial, you will use the IBM® [Granite-3-8B-Instruct model](https://www.ibm.com/granite) now available on watsonx.ai™ to perform custom function calling.  

Traditional [large language models (LLMs)](https://www.ibm.com/topics/large-language-models), like the OpenAI GPT-4 (generative pre-trained transformer) model available through ChatGPT, and the IBM Granite™ models that we'll use in this tutorial, are limited in their knowledge and reasoning. They produce their responses based on the data used to train them and are difficult to adapt to personalized user queries. To obtain the missing information, these [generative AI](https://www.ibm.com/topics/generative-ai) models can integrate external tools within the function calling. This method is one way to avoid fine-tuning a foundation model for each specific use-case. The function calling examples in this tutorial will implement external [API](https://www.ibm.com/topics/api) calls.

The Granite-3-8B-Instruct model and tokenizer use [natural language processing (NLP)](https://www.ibm.com/topics/natural-language-processing) to parse query syntax. In addition, the models use function descriptions and function parameters to determine the appropriate tool calls. Key information is then extracted from user queries to be passed as function arguments.

[![Open YouTube video](https://img.youtube.com/vi/cjCYcTPryw8/0.jpg)](https://www.youtube.com/watch?v=cjCYcTPryw8)

This recipe is also available on [IBM.com](https://www.ibm.com/think/tutorials/granite-function-calling).


# Steps

## Step 1. Set up your environment

While you can choose from several tools, this tutorial is best suited for a Jupyter Notebook. Jupyter Notebooks are widely used within data science to combine code with various data sources such as text, images and data visualizations.

You can run this notebook in [Colab](https://colab.research.google.com/), or download it to your system and [run the notebook locally](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started_with_Jupyter_Locally/Getting_Started_with_Jupyter_Locally.md).

To avoid Python package dependency conflicts, we recommend setting up a [virtual environment](https://docs.python.org/3/library/venv.html).  

Note, this notebook is compatible with Python 3.11.10 and well as Python 3.10.12, the default in Colab at the time of publishing this tutorial. To check your python version, you can run the `!python --version` command in a code cell.

## Step 2. Set up a watsonx.ai Runtime instance and API key.

1. Create a [watsonx.ai Runtime](https://cloud.ibm.com/catalog/services/watsonxai-runtime) service instance (select your appropriate region and choose the Lite plan, which is a free instance).

2. Generate an application programming interface [(API) key](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html).

3. Associate the watsonx.ai Runtime service instance to the project that you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html).

## Step 3. Install and import relevant libraries and set up your credentials

We'll need a few libraries and modules for this tutorial. Make sure to import the following ones; if they're not installed, you can resolve this with a quick pip install.

In [ ]:
# installations
! pip install -q "git+https://github.com/ibm-granite-community/utils.git" \
    langchain-ibm \
    transformers

In [ ]:
#imports
import requests
import ast
import re

from transformers import AutoTokenizer

Next, we can prepare our environment by setting the model ID for the `granite-3-8b-instruct` model, and the tokenizer for the same Granite model.

In [ ]:
MODEL_ID = "ibm/granite-3-2-8b-instruct"

TOKENIZER = AutoTokenizer.from_pretrained("ibm-granite/granite-3.2-8b-instruct")

The `get_stock_price` function in this tutorial requires an `AV_STOCK_API_KEY` key. To generate a free `AV_STOCK_API_KEY`, please visit the [Alpha Vantage website](https://www.alphavantage.co/support/#api-key).

Secondly, the `get_current_weather` function requires a `WEATHER_API_KEY`. To generate one, please [create an account](https://home.openweathermap.org/users/sign_up). Upon creating an account, select the "API Keys" tab to display your free key.

**Store these private keys in a separate `.env` file in the same level of your directory as this notebook.**

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var

AV_STOCK_API_KEY = get_env_var("AV_STOCK_API_KEY", "none")

WEATHER_API_KEY = get_env_var("WEATHER_API_KEY", "none")

## Step 4. Define the functions

We can now define our functions. The function's docstring and type information are important for generating the proper tool information.

In this tutorial, the `get_stock_price` function uses the Stock Market Data API available through Alpha Vantage.

In [ ]:
def get_stock_price(ticker: str, date: str) -> dict:
    """
    Retrieves the lowest and highest stock prices for a given ticker and date.

    Args:
        ticker: The stock ticker symbol, e.g., "IBM".
        date: The date in "YYYY-MM-DD" format for which you want to get stock prices.

    Returns:
        A dictionary containing the low and high stock prices on the given date.
    """
    print(f"Getting stock price for {ticker} on {date}")
    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={AV_STOCK_API_KEY}"
        stock_data = requests.get(stock_url)
        stock_low = stock_data.json()["Time Series (Daily)"][date]["3. low"]
        stock_high = stock_data.json()["Time Series (Daily)"][date]["2. high"]
        return {
            "low": stock_low,
            "high": stock_high
        }
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return {
            "low": "none",
            "high": "none"
        }

The `get_current_weather` function retrieves the real-time weather in a given location using the Current Weather Data API via [OpenWeather](https://openweathermap.org/api).

In [ ]:
def get_current_weather(location: str) -> dict:
    """
    Fetches the current weather for a given location (default: San Francisco).

    Args:
        location: The name of the city for which to retrieve the weather information.

    Returns:
        A dictionary containing weather information such as temperature, weather description, and humidity.
    """
    print(f"Getting current weather for {location}")

    try:
        # API request to fetch weather data
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
        weather_data = requests.get(weather_url)
        data = weather_data.json()
        # Extracting relevant weather details
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]

        # Returning weather details
        return {
            "description": weather_description,
            "temperature": temperature,
            "humidity": humidity
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {
            "description": "none",
            "temperature": "none",
            "humidity": "none"
        }

## Step 5. Set up the API request

Now that our functions are defined, we can create a function that generates a watsonx API request for the provided instructions the watsonx API endpoint. We will use this function each time we make a request.

Please store your `WATSONX_PROJECT_ID` and `WATSONX_APIKEY` in a separate `.env` file in the same level of your directory as this notebook. Your `WATSONX_URL` will be dependent on your region. Reference the [documentation](https://cloud.ibm.com/apidocs/watsonx-ai) to determine the appropriate endpoint for you.

In [ ]:
from langchain_ibm import WatsonxLLM

def make_api_request(instructions: str) -> str:
    model_parameters = {
        "decoding_method": "greedy",
        "max_new_tokens": 200,
        "repetition_penalty": 1.05,
        "stop_sequences": [TOKENIZER.eos_token]
    }
    model = WatsonxLLM(
        model_id=MODEL_ID,
        url= get_env_var("WATSONX_URL"),
        apikey=get_env_var("WATSONX_APIKEY"),
        project_id=get_env_var("WATSONX_PROJECT_ID"),
        params=model_parameters
    )
    response = model.invoke(instructions)
    return response


Next, we can create a list of available functions. Here, we declare our function definitions that require the function names, descriptions, parameters and required properties.

In [ ]:
from transformers.utils import get_json_schema

tools = [get_json_schema(tool)["function"] for tool in (get_stock_price, get_current_weather)]
tools

## Step 6. Perform function calling


### Step 6a. Calling the get_stock_price function

To prepare for the API requests, we must set our `query` used in the tokenizer chat template.

In [ ]:
query = "What were the IBM stock prices on October 7, 2024?"

Applying a chat template is useful for breaking up long strings of texts into one or more messages with corresponding labels. This allows the LLM to process the input in a format that it expects. Because we want our output to be in a string format, we can set the `tokenize` parameter to false. The `add_generation_prompt` can be set to true in order to append the tokens indicating the beginning of an assistant message to the output. This will be useful when generating chat completions with the model.

In [ ]:
conversation = [
    {"role": "system","content": "You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required."},
    {"role": "user", "content": query },
]

instruction_1 = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, tokenize=False, add_generation_prompt=True)
instruction_1

Now, we can call the `make_api_request` function and pass the instructions we generated.

In [ ]:
data_1 = make_api_request(instruction_1)
data_1

As you can see by the function name in the JSON object produced by the model, the appropriate `get_stock_price` tool use was selected from the set of functions. To run the function, let's extract relevant information from the output. With the function name and arguments extracted, we can call the function. To call the function using its name as a string, we can use the `globals()` function.

In [ ]:
def tool_call(llm_response: str):
    tool_request = ast.literal_eval(re.search("({.+})", llm_response).group(0))
    tool_name = tool_request["name"]
    tool_arguments = tool_request["arguments"]
    tool_response = globals()[tool_name](**tool_arguments)
    return tool_response

Get the response from the requested tool.

In [ ]:
tool_response = tool_call(data_1)
tool_response

The function successfully retrieved the requested stock price. To generate a synthesized final response, we can pass another prompt to the Granite model along with the information collected from function calling.

In [ ]:
conversation2 = conversation + [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Display the tool response in natural language." },
    {"role": "tool_response", "content": str(tool_response) },
]

instruction_2 = TOKENIZER.apply_chat_template(conversation=conversation2, tools=tools, tokenize=False, add_generation_prompt=True)
data_2 = make_api_request(instruction_2)
data_2

### Step 6b. Calling the get_current_weather function

As our next query, let’s inquire about the current weather in San Francisco. We can follow the same steps as in Step 5a by adjusting the query.

In [ ]:
query = "What is the current weather in San Francisco?"

In [ ]:
conversation = [
    {"role": "system","content": "You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required."},
    {"role": "user", "content": query },
]

instruction_1 = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, tokenize=False, add_generation_prompt=True)
instruction_1

In [ ]:
data_1 = make_api_request(instruction_1)
data_1

Once again, the model decides the appropriate tool choice, in this case `get_current_weather`, and extracts the location correctly. Now, let's call the function with the argument generated by the model.

In [ ]:
tool_response = tool_call(data_1)
tool_response

The function response correctly describes the current weather in San Francisco. Lastly, let's generate a synthesized final response with the results of this function call.

In [ ]:
conversation2 = conversation + [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Display the tool response in natural language." },
    {"role": "tool_response", "content": str(tool_response) },
]

instruction_2 = TOKENIZER.apply_chat_template(conversation=conversation2, tools=tools, tokenize=False, add_generation_prompt=True)
data_2 = make_api_request(instruction_2)
data_2

## Summary

In this tutorial, you built custom functions and used the Granite-3-8B-Instruct model to determine which function to call based on  key information from user queries. With this information, you called the function with the arguments as stated in the model response. These function calls produce the expected output. Finally, you called the Granite-3-8B-Instruct model again to synthesize the information returned by the functions.